# BI Exercises

## Set Up

In [2]:
%load_ext sql

In [3]:
%config SqlMagic.autopandas=True

In [4]:
%config SqlMagic.displaycon=False

In [5]:
%sql sqlite:///transactions.db

In [6]:
%%sql
SELECT *
FROM transactions
LIMIT 5

Done.


,transaction_id,payment_method,seller_id,buyer_id,transaction_date,product_price
0,1294738,1,5107765,5007941,2022-05-11 15:46:38,1410.0
1,585369,4,4205200,4250752,2021-03-27 15:01:28,800.0
2,882933,4,581926,4695227,2021-10-20 14:45:11,800.0
3,1145325,3,536932,4865235,2022-03-06 17:37:04,1616.0
4,762483,5,432561,4186489,2021-08-06 11:33:12,840.0


In [7]:
%%sql
SELECT *
FROM users
LIMIT 5

Done.


,user_id,register_date
0,5042412,2022-03-01 21:26:27
1,707930,2020-09-29 08:33:26
2,5283835,2022-06-23 11:18:09
3,4195678,2021-02-24 22:12:15
4,5370530,2022-08-29 13:53:33


## Preguntas

### Agregaciones simples

#### Qué procentaje de usuarias registradas han comprado al menos un producto?

#### Qué procentaje de compradoras no están registradas?

In [8]:
%%sql buyer_transactions <<
SELECT user_id, COUNT(DISTINCT transaction_id) AS transactions
FROM users u 
LEFT JOIN transactions t 
ON u.user_id = t.buyer_id
GROUP BY user_id

Done.
Returning data to local variable buyer_transactions


In [9]:
buyer_transactions['transactions'] = list(map(lambda x: 1 if x> 0 else 0, buyer_transactions['transactions']))
buyer_transactions['transactions'].mean()*100

0.9946666666666666

In [10]:
%%sql buyers_registered <<
SELECT buyer_id, COUNT(DISTINCT register_date) AS registered
FROM users u 
RIGHT JOIN transactions t 
ON u.user_id = t.buyer_id
GROUP BY buyer_id

(sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: SELECT buyer_id, COUNT(DISTINCT register_date) AS registered
FROM users u 
RIGHT JOIN transactions t 
ON u.user_id = t.buyer_id
GROUP BY buyer_id]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


### Retención externa

#### Qué procentaje de usuarias han comprado al menos un producto el mes en el que se registró?

In [12]:
%%sql test <<
SELECT
    user_id,
    strftime('%Y-%m-01', register_date) AS resgister_month,
    strftime('%Y-%m-01', transaction_date) AS transaction_month,
    strftime('%Y-%m-01', transaction_date) - strftime('%Y-%m-01', register_date) AS datediff
FROM users u
LEFT JOIN transactions t
ON u.user_id = t.buyer_id
WHERE transaction_id IS NOT NULL

Done.
Returning data to local variable test


In [13]:
test

,user_id,resgister_month,transaction_month,datediff
0,5007941,2022-02-01,2022-05-01,0
1,4634379,2021-09-01,2021-09-01,0
2,5106839,2022-03-01,2022-04-01,0
3,4030474,2020-11-01,2021-03-01,1
4,4860976,2021-12-01,2022-09-01,1
...,...,...,...,...
1357,691573,2020-09-01,2021-08-01,1
1358,4071450,2020-12-01,2021-03-01,1
1359,4003160,2020-10-01,2021-12-01,1
1360,4252587,2021-03-01,2021-03-01,0


In [ ]:
test[test['resgister_month'] == test['transaction_month']

#### Cual fue el mes en el que las usuarias que se registraron convirtieron mejor a sellers? (Vendieron más items). Se mantuvo esta tendencia?

### Retención interna

#### Ha habido algún mes en el que las usuarias que compraron ese mes no volvieron a comprar el mes siguiente en mayor medida que el resto?

#### Que porcentaje de las compras cada mes están realizadas por usuarias que nunca han comprado? Y por usuarias con más de 3 meses desde su primera compra?